In [1]:
# helper functions

# get http request headers
def getHeaders():
    headers = {
        "apikey":akey,
        "accept":"application/json"
    }
    return headers


# GET /property/address via zip code
def fetch(url, params):
    headers = getHeaders()
    res = requests.get(url, headers=headers, params=params).json()
    return res


# show load bar and wait for next request
def wait(t):    
    sys.stdout.write('.')
    sys.stdout.flush()
    time.sleep(t)
    

def f(x):
    if x >= 10:
        return 10
    elif x <= 0:
        return 1
       
    else:
        return x 
    

def g(x):
    if x >= 5:
        return 10
    elif x <= 0:
        return 1
    else:
        return 5 
    
    
def h(x):
    return x['pt_rankStars'] + x['pt_rankStatewidePercentage'] + x['pt_averageStandardScore'] + x['pt_rankMovement'] + x['pt_pupilTeacherRatio']


def i(x):
    return (x['high_score'] + x['middle_score'] + x['elementary_score']) / 15


def k(x):
    if x >= 100:
        return 10
    elif x == 100:
        return 7.5
    elif x >= 50 and  x < 100:
        return 5
    elif x > 0 and  x < 50:
        return 1
    else:
        return 0


def l(x):
    return (x['score_grocery_stores'] + x['score_avg_police'] + x['score_station'] + x['score_avg_museum'])

In [2]:
# Dependencies
import sys
import time
import json
import requests
import importlib
import numpy as np
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt
from config import gkey, zkey, zws_id, akey

In [3]:
# get list of zipcodes
%run ./write_zips.ipynb
import zipcodes
importlib.reload(zipcodes)
required_zips = zipcodes.ziplist

Target city: atlanta
Search radius: 1


In [4]:
required_zips

['30379',
 '30387',
 '30389',
 '30390',
 '30399',
 '31120',
 '31197',
 '31198',
 '31199',
 '30312',
 '30334',
 '30302',
 '30321',
 '30343',
 '30353',
 '30370',
 '30392',
 '30394',
 '30303']

In [4]:
# get lat, lng coords from ziplist
url = "https://maps.googleapis.com/maps/api/geocode/json?"
response = []
latlngs = []

for z in required_zips:
    params = {
        "address":z,
        "key":gkey
    }

    res = requests.get(url, params=params).json()
    response.append({"response":res, "zipcode": z})

for r in response:
    try:
        zc = r['zipcode']
        lat = r['response']['results'][0]['geometry']['location']['lat']
        lng = r['response']['results'][0]['geometry']['location']['lng']
        
        latlngs.append({
            "lat":lat,
            "lng":lng,
            "zipcode":zc
        })

    except:
        continue

In [5]:
len(latlngs)

8

# Neighbourhood Analysis

In [6]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
ttypes = ['grocery_or_supermarket', 'police', 'transit_station', 'museum']

places_data = []

for l in latlngs:
    curzip = l['zipcode']
    curlatlng = f"{l['lat']},{l['lng']}"
    
    for i,t in enumerate(ttypes):
        curtype = t
        print(f"zipcode: {curzip}, latlng: {curlatlng}, target_type: {curtype}")
        
        params = {
            "target_search": curtype,
            "location": curlatlng,
            "radius": 1609, # TODO: pull this in for dynamic search radius
            "type": curtype,
            "key":gkey
        }

        response = requests.get(base_url, params=params).json()

        places_data.append({
            "zipcode":curzip,
            "latlng":curlatlng,
            "type":curtype,
            "response":response
        })

zipcode: 30387, latlng: 28.048427,-82.7376544, target_type: grocery_or_supermarket
zipcode: 30387, latlng: 28.048427,-82.7376544, target_type: police
zipcode: 30387, latlng: 28.048427,-82.7376544, target_type: transit_station
zipcode: 30387, latlng: 28.048427,-82.7376544, target_type: museum
zipcode: 30312, latlng: 33.7440941,-84.377861, target_type: grocery_or_supermarket
zipcode: 30312, latlng: 33.7440941,-84.377861, target_type: police
zipcode: 30312, latlng: 33.7440941,-84.377861, target_type: transit_station
zipcode: 30312, latlng: 33.7440941,-84.377861, target_type: museum
zipcode: 30334, latlng: 33.7476898,-84.38804329999999, target_type: grocery_or_supermarket
zipcode: 30334, latlng: 33.7476898,-84.38804329999999, target_type: police
zipcode: 30334, latlng: 33.7476898,-84.38804329999999, target_type: transit_station
zipcode: 30334, latlng: 33.7476898,-84.38804329999999, target_type: museum
zipcode: 30302, latlng: 33.75,-84.39999999999999, target_type: grocery_or_supermarket
zip

In [37]:
# pprint(len(places_data))
pd_df = pd.DataFrame(places_data)
for i,r in enumerate(pd_df['response']):
    for s in r['results']:
        pprint(s)

{'geometry': {'location': {'lat': 28.0500507, 'lng': -82.73574579999999},
              'viewport': {'northeast': {'lat': 28.0518385802915,
                                         'lng': -82.7347916},
                           'southwest': {'lat': 28.0491406197085,
                                         'lng': -82.7386084}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/shopping-71.png',
 'id': 'aa0fcec7e06130374358567d94af1427bb6bf1d9',
 'name': 'Publix Super Market at Seabreeze Plaza Shopping Center',
 'opening_hours': {'open_now': True},
 'photos': [{'height': 2988,
             'html_attributions': ['<a '
                                   'href="https://maps.google.com/maps/contrib/107983370411696368436/photos">Melinda '
                                   'Russek</a>'],
             'photo_reference': 'CmRaAAAA8YEYKi7RwktdwiWHlm-o2X_8VzBWpbPsEY-6pZ1SgdDpmSkYSUctgzpgjtV72Bv-X9WsTdbHsE3QCeAZi8Hcwq5O3l3TWeqVLwuRewL0eo-MWvjWJsED2KbZM36HQJxqEhAUrtX7wt3ckbob286GvX2RGh

 'photos': [{'height': 3984,
             'html_attributions': ['<a '
                                   'href="https://maps.google.com/maps/contrib/104554058328932267255/photos">Xander '
                                   'Dawson</a>'],
             'photo_reference': 'CmRaAAAA5tBCzkNb3g3fqnYmsTX9KnckDdFc39txABASCgV8fhtJ1xP56e_88TIB7fl26N_NDDywOzy3Fo7PTYczCN0FgEK10Nf389fZgnkpxvmuEWHKeKTXrYStkaNLHUJ-SJN1EhCn_67da2RH8n0Fcj6sujJJGhTiGT4n4OL1kMxudRbjNRTvTyCjpw',
             'width': 2988}],
 'place_id': 'ChIJeTgTkoYD9YgRO1G5qF8XTYI',
 'plus_code': {'compound_code': 'QJ36+J9 Atlanta, Georgia, United States',
               'global_code': '865QQJ36+J9'},
 'rating': 3,
 'reference': 'ChIJeTgTkoYD9YgRO1G5qF8XTYI',
 'scope': 'GOOGLE',
 'types': ['police', 'point_of_interest', 'establishment'],
 'vicinity': '1 Peachtree Center Avenue Southeast, Atlanta'}
{'geometry': {'location': {'lat': 33.7502577, 'lng': -84.3862663},
              'viewport': {'northeast': {'lat': 33.7516869802915,
        

              'viewport': {'northeast': {'lat': 33.7349137802915,
                                         'lng': -84.36970851970848},
                           'southwest': {'lat': 33.7322158197085,
                                         'lng': -84.37240648029149}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/museum-71.png',
 'id': '4d91ae8c9355319c64cfe299f3a7ac5931ab51ff',
 'name': 'Zoolojico',
 'photos': [{'height': 2576,
             'html_attributions': ['<a '
                                   'href="https://maps.google.com/maps/contrib/115609305051089277470/photos">Jan '
                                   'Hernandez</a>'],
             'photo_reference': 'CmRaAAAAqnarya8XQEcT0eSSBWsXnY8CAwFFGR6_46VfhuTmXnT68DDZXDNM6Y59B4VH9DavOs1VVA-yjlYxgcmb3Y5Z5KJmeStk9XRfp27xIB3YGwXnrTrdOJ0uQE97gb4MDXxzEhBxtQS376v-Gs_MFau7SFTwGhRiTYt-34YoU7zaMyLA4Kh-93nFOQ',
             'width': 1932}],
 'place_id': 'ChIJDe7olsED9YgRQb5AyQbbymk',
 'plus_code': {'compound_code': 'PJMH+CH 

 'place_id': 'ChIJC2XbQYED9YgR8KC5tasG6vI',
 'plus_code': {'compound_code': 'QJ35+F6 Atlanta, Georgia, United States',
               'global_code': '865QQJ35+F6'},
 'rating': 2,
 'reference': 'ChIJC2XbQYED9YgR8KC5tasG6vI',
 'scope': 'GOOGLE',
 'types': ['bus_station',
           'transit_station',
           'point_of_interest',
           'establishment'],
 'vicinity': '30 Alabama Street Southwest, Atlanta'}
{'geometry': {'location': {'lat': 33.7504165, 'lng': -84.3891727},
              'viewport': {'northeast': {'lat': 33.7518642802915,
                                         'lng': -84.38821136970849},
                           'southwest': {'lat': 33.7491663197085,
                                         'lng': -84.39090933029149}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/bus-71.png',
 'id': 'bf786c6240fa2e4922b6eea4a5eb8c5250df3de5',
 'name': 'MARTA',
 'place_id': 'ChIJJ7myR4QD9YgR4asdUoRIVWk',
 'plus_code': {'compound_code': 'QJ26+58 Atlanta, Georgia, Un

{'geometry': {'location': {'lat': 33.7517387, 'lng': -84.3920012},
              'viewport': {'northeast': {'lat': 33.75284858029151,
                                         'lng': -84.3901914697085},
                           'southwest': {'lat': 33.75015061970851,
                                         'lng': -84.3928894302915}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/police-71.png',
 'id': 'd628e1b99da5bbf9ac9a6120b51c1dd7a7e2ae5e',
 'name': 'Fulton County Police Department',
 'photos': [{'height': 1960,
             'html_attributions': ['<a '
                                   'href="https://maps.google.com/maps/contrib/112853533286606521351/photos">mike '
                                   'kricke</a>'],
             'photo_reference': 'CmRaAAAAxGnc4bUMcCQpBwTk4UXMDpHPk5TzRx9x5A82Dm9AUrfjoFAItEYBGTD22xAv0yJohRCcNvyVR6oA_jGffTIpUhB8FBAUmagoBPC0jHwTc2aA9t_V2uSQphfLEG9CMmGgEhChDxg2Ii7D931TJD0qxmSBGhTDP3_w7khD18oLKzJ3csYC3sA04A',
             'width': 3171}]

...

              'viewport': {'northeast': {'lat': 33.7522947802915,
                                         'lng': -84.3897360197085},
                           'southwest': {'lat': 33.7495968197085,
                                         'lng': -84.3924339802915}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/police-71.png',
 'id': 'c7bf31ca1683415928d349aaeb2974003091c943',
 'name': 'Superior Court-Criminal',
 'place_id': 'ChIJ6UGBiIMD9YgRyAN70stRgVk',
 'plus_code': {'compound_code': 'QJ25+8J Atlanta, Georgia, United States',
               'global_code': '865QQJ25+8J'},
 'reference': 'ChIJ6UGBiIMD9YgRyAN70stRgVk',
 'scope': 'GOOGLE',
 'types': ['local_government_office',
           'police',
           'point_of_interest',
           'establishment'],
 'vicinity': '136 Pryor Street Southwest # C515, Atlanta'}
{'geometry': {'location': {'lat': 33.7503675, 'lng': -84.3912916},
              'viewport': {'northeast': {'lat': 33.7518105302915,
                         

           'point_of_interest',
           'establishment'],
 'vicinity': 'United States'}
{'geometry': {'location': {'lat': 33.75024529999999, 'lng': -84.3883063},
              'viewport': {'northeast': {'lat': 33.75168813029149,
                                         'lng': -84.38706286970849},
                           'southwest': {'lat': 33.7489901697085,
                                         'lng': -84.3897608302915}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png',
 'id': 'ead4a882bb3cc3ec7af8a246635fee383b2c5dd7',
 'name': 'Washington St @ MLK Jr Dr',
 'place_id': 'ChIJgVvBqoUD9YgR9HTL1G9u-LE',
 'plus_code': {'compound_code': 'QJ26+3M Atlanta, Georgia, United States',
               'global_code': '865QQJ26+3M'},
 'reference': 'ChIJgVvBqoUD9YgR9HTL1G9u-LE',
 'scope': 'GOOGLE',
 'types': ['transit_station', 'point_of_interest', 'establishment'],
 'vicinity': 'United States'}
{'geometry': {'location': {'lat': 33.7520195, 'lng': -84.39

{'geometry': {'location': {'lat': 33.7504957, 'lng': -84.3904244},
              'viewport': {'northeast': {'lat': 33.7517643302915,
                                         'lng': -84.3889289197085},
                           'southwest': {'lat': 33.7490663697085,
                                         'lng': -84.39162688029151}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/police-71.png',
 'id': 'f2ecaedb59991f120887dd310f67329fe5dc10da',
 'name': 'Sheriff-Superior Court Office',
 'place_id': 'ChIJ43mbYoQD9YgRCgx-iEFAUzE',
 'plus_code': {'compound_code': 'QJ25+5R Atlanta, Georgia, United States',
               'global_code': '865QQJ25+5R'},
 'reference': 'ChIJ43mbYoQD9YgRCgx-iEFAUzE',
 'scope': 'GOOGLE',
 'types': ['local_government_office',
           'police',
           'point_of_interest',
           'establishment'],
 'vicinity': '185 Central Avenue Southwest # 900, Atlanta'}
{'geometry': {'location': {'lat': 33.750364, 'lng': -84.3905553},
              'vi

 'plus_code': {'compound_code': 'QJ48+4Q Atlanta, Georgia, United States',
               'global_code': '865QQJ48+4Q'},
 'rating': 4.6,
 'reference': 'ChIJp3TRAYkD9YgROwWFdqNufKg',
 'scope': 'GOOGLE',
 'types': ['museum', 'point_of_interest', 'establishment'],
 'vicinity': '135 Auburn Avenue Northeast, Atlanta'}
{'geometry': {'location': {'lat': 33.7601908, 'lng': -84.395534},
              'viewport': {'northeast': {'lat': 33.7617714302915,
                                         'lng': -84.3940903197085},
                           'southwest': {'lat': 33.7590734697085,
                                         'lng': -84.3967882802915}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/museum-71.png',
 'id': 'd5a595fa7de13ebfb7758d5e0ce5b1f3bf0febed',
 'name': 'College Football Hall of Fame',
 'opening_hours': {'open_now': True},
 'photos': [{'height': 3024,
             'html_attributions': ['<a '
                                   'href="https://maps.google.com/maps/c

{'geometry': {'location': {'lat': 33.766245, 'lng': -84.38750399999999},
              'viewport': {'northeast': {'lat': 33.76759398029149,
                                         'lng': -84.3861550197085},
                           'southwest': {'lat': 33.7648960197085,
                                         'lng': -84.3888529802915}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png',
 'id': '4d1a25cd088958e59ee0fbc756f22979cce8458d',
 'name': 'MegaBus Station',
 'place_id': 'ChIJRd4PW3oE9YgRiAZhvGbE_pM',
 'plus_code': {'compound_code': 'QJ86+FX Atlanta, Georgia, United States',
               'global_code': '865QQJ86+FX'},
 'rating': 3.6,
 'reference': 'ChIJRd4PW3oE9YgRiAZhvGbE_pM',
 'scope': 'GOOGLE',
 'types': ['transit_station', 'point_of_interest', 'establishment'],
 'vicinity': 'United States'}
{'geometry': {'location': {'lat': 33.766447, 'lng': -84.3873261},
              'viewport': {'northeast': {'lat': 33.7677978302915,
              

In [ ]:
# # extract graocery stores, police stations, museums, transit stations and assign scores
# averagegrocery = placedata_df_pd["grocery_stores"].mean()
# avergaepolice = placedata_df_pd["police"].mean()
# avergaemuseum = placedata_df_pd["museum"].mean()
# avergaetransit_station = placedata_df_pd["transit_station"].mean()

# # Assign scores 
# placedata_df_pd["pg_avg_grocery_stores"] = (placedata_df_pd["grocery_stores"]/averagegrocery*100).fillna(0.0).astype(int)
# placedata_df_pd["pg_avg_police"] = (placedata_df_pd["police"]/avergaepolice*100 ).fillna(0.0).astype(int)
# placedata_df_pd["pg_avg_museum"] = (placedata_df_pd["museum"]/avergaemuseum*100 ).fillna(0.0).astype(int)
# placedata_df_pd["pg_avg_transit_station"] = placedata_df_pd["transit_station"]/avergaetransit_station*100

In [ ]:
# placedata_df_pd["score_grocery_stores"] = placedata_df_pd.apply(lambda row: k(row['pg_avg_grocery_stores']), axis=1)
# placedata_df_pd["score_avg_police"] = placedata_df_pd.apply(lambda row: k(row['pg_avg_police']), axis=1)
# placedata_df_pd["score_station"] = placedata_df_pd.apply(lambda row: k(row['pg_avg_transit_station']), axis=1)
# placedata_df_pd["score_avg_museum"] = placedata_df_pd.apply(lambda row: k(row['pg_avg_museum']), axis=1)
# placedata_df_pd["total_score"] = placedata_df_pd.apply(lambda row: l(row), axis=1)
# placedata_df_pd

# School Analysis

In [ ]:
# Inititalize arrays for all the various parameters returned from response of call that is needed
postal = []
schoolLevel = []
rankStars = []
rankStatewidePercentage = []
averageStandardScore = []
rankMovement = []
pupilTeacherRatio = []

In [ ]:
school_base_url="https://api.schooldigger.com/v1.1/schools?"

for zip in required_zips:
    params = {
        "appID":'cdcbd9a2',
        "appKey":'f21463a388bc041e25afa5dc32fb5014',
        "st":'GA',
        "zip":zip
    }

    # assemble url and make API request
    response = requests.get(school_base_url, params=params).json()
    response_df = pd.DataFrame(response)
    
    numberofresults = len(response_df)
    for i in range(numberofresults):
        schoolLevel.append(response_df["schoolList"][i].get("schoolLevel",np.nan))
        rankMovement.append(response_df["schoolList"][i].get("rankMovement",np.nan))
        postal.append(response_df["schoolList"][i]["address"].get("zip",np.nan))            

        try:
            rankStars.append(response_df["schoolList"][i]["rankHistory"][0].get("rankStars",np.nan))
            rankStatewidePercentage.append(response_df["schoolList"][i]["rankHistory"][0].get("rankStatewidePercentage",np.nan))
            averageStandardScore.append(response_df["schoolList"][i]["rankHistory"][0].get("averageStandardScore",np.nan))

        except (KeyError, IndexError, TypeError):
            rankStars.append(np.nan)
            rankStatewidePercentage.append(np.nan)
            averageStandardScore.append(np.nan)
                
        try:    
            pupilTeacherRatio.append(response_df["schoolList"][i]["schoolYearlyDetails"][0].get("pupilTeacherRatio",np.nan))
    
        except (KeyError, IndexError, TypeError):
            pupilTeacherRatio.append(np.nan)

        time.sleep(30)

In [ ]:
# create a data frame from all schools
school_dict = {
        "postal":postal,
        "schoolLevel":schoolLevel,
        "rankStars":rankStars,
        "rankStatewidePercentage":rankStatewidePercentage,
        "averageStandardScore":averageStandardScore,
        "rankMovement":rankMovement,
        "pupilTeacherRatio":pupilTeacherRatio
}

school_dict_pd = pd.DataFrame(school_dict)
school_dict_pd.replace(np.nan, 0, inplace=True)
school_dict_pd

In [ ]:
# seperate middle, elementary and high and assign scores
school_elementary_pd = school_dict_pd[school_dict_pd['schoolLevel'] == 'Elementary']
school_elementary_pd = school_elementary_pd.copy()
averagerankStars = school_elementary_pd["rankStars"].mean()

avergaerankStatewidePercentage = school_elementary_pd["rankStatewidePercentage"].mean()
avergaeaverageStandardScore = school_elementary_pd["averageStandardScore"].mean()
avergaerankMovement = school_elementary_pd["rankMovement"].mean()
avergaepupilTeacherRatio = school_elementary_pd["pupilTeacherRatio"].mean()

# Assign scores
school_elementary_pd["pg_avg_rankStars"] = (school_elementary_pd["rankStars"]/averagerankStars *10).fillna(0.0).astype(int)
school_elementary_pd["pg_avg_rankStatewidePercentage"] = (school_elementary_pd["rankStatewidePercentage"]/avergaerankStatewidePercentage *10).fillna(0.0).astype(int)
school_elementary_pd["pg_avg_averageStandardScore"] = (school_elementary_pd["averageStandardScore"]/avergaeaverageStandardScore *10).fillna(0.0).astype(int)
school_elementary_pd["pg_avg_rankMovement"] = school_elementary_pd["rankMovement"]/avergaerankMovement *10
school_elementary_pd["pg_avg_pupilTeacherRatio"] = (school_elementary_pd["pupilTeacherRatio"]/avergaepupilTeacherRatio *10).fillna(0.0).astype(int)

school_elementary_pd['pt_rankStars'] = school_elementary_pd.apply(lambda row: f(row['pg_avg_rankStars']), axis=1)
school_elementary_pd['pt_rankStatewidePercentage'] = school_elementary_pd.apply(lambda row: f(row['pg_avg_rankStatewidePercentage']), axis=1)
school_elementary_pd['pt_averageStandardScore'] = school_elementary_pd.apply(lambda row: f(row['pg_avg_averageStandardScore']), axis=1)
school_elementary_pd['pt_rankMovement'] = school_elementary_pd.apply(lambda row: g(row['rankMovement']), axis=1)
school_elementary_pd['pt_pupilTeacherRatio'] = school_elementary_pd.apply(lambda row: f(row['pg_avg_pupilTeacherRatio']), axis=1)
school_elementary_pd['total_points'] = school_elementary_pd.apply(lambda row: h(row),axis=1)
school_elementary_points = school_elementary_pd[['postal','total_points']]
elementary_groupby = school_elementary_points.groupby('postal').mean()

# elementary_groupby
school_elementary_pd

In [ ]:
# seperate middle, middle and high and assign scores
school_middle_pd = school_dict_pd[school_dict_pd['schoolLevel']=='Middle']
school_middle_pd = school_middle_pd.copy()
averagerankStars = school_middle_pd["rankStars"].mean()
avergaerankStatewidePercentage = school_middle_pd["rankStatewidePercentage"].mean()
avergaeaverageStandardScore = school_middle_pd["averageStandardScore"].mean()
avergaerankMovement = school_middle_pd["rankMovement"].mean()
avergaepupilTeacherRatio = school_middle_pd["pupilTeacherRatio"].mean()

# Assign scores
school_middle_pd["pg_avg_rankStars"] = (school_middle_pd["rankStars"]/averagerankStars *10).fillna(0.0).astype(int)
school_middle_pd["pg_avg_rankStatewidePercentage"] = (school_middle_pd["rankStatewidePercentage"]/avergaerankStatewidePercentage *10).fillna(0.0).astype(int)
school_middle_pd["pg_avg_averageStandardScore"] = (school_middle_pd["averageStandardScore"]/avergaeaverageStandardScore *10).fillna(0.0).astype(int)
school_middle_pd["pg_avg_rankMovement"] = school_middle_pd["rankMovement"]/avergaerankMovement *10
school_middle_pd["pg_avg_pupilTeacherRatio"] = (school_middle_pd["pupilTeacherRatio"]/avergaepupilTeacherRatio *10).fillna(0.0).astype(int)

school_middle_pd['pt_rankStars'] = school_middle_pd.apply(lambda row: f(row['pg_avg_rankStars']), axis=1)
school_middle_pd['pt_rankStatewidePercentage'] = school_middle_pd.apply(lambda row: f(row['pg_avg_rankStatewidePercentage']), axis=1)
school_middle_pd['pt_averageStandardScore'] = school_middle_pd.apply(lambda row: f(row['pg_avg_averageStandardScore']), axis=1)
school_middle_pd['pt_rankMovement'] = school_middle_pd.apply(lambda row: g(row['rankMovement']), axis=1)
school_middle_pd['pt_pupilTeacherRatio'] = school_middle_pd.apply(lambda row: f(row['pg_avg_pupilTeacherRatio']), axis=1)
school_middle_pd['total_points'] = school_middle_pd.apply(lambda row: h(row),axis=1)

school_middle_points=school_middle_pd[['postal','total_points']]
middle_groupby=school_middle_points.groupby('postal').mean()
middle_groupby

In [ ]:
# seperate high, high and high and assign scores
school_high_pd = school_dict_pd[school_dict_pd['schoolLevel']=='High']
school_high_pd = school_high_pd.copy()
averagerankStars = school_high_pd["rankStars"].mean()
avergaerankStatewidePercentage = school_high_pd["rankStatewidePercentage"].mean()
avergaeaverageStandardScore = school_high_pd["averageStandardScore"].mean()
avergaerankMovement = school_high_pd["rankMovement"].mean()
avergaepupilTeacherRatio = school_high_pd["pupilTeacherRatio"].mean()

# Assign scores
school_high_pd["pg_avg_rankStars"] = (school_high_pd["rankStars"]/averagerankStars *10).fillna(0.0).astype(int)
school_high_pd["pg_avg_rankStatewidePercentage"] = (school_high_pd["rankStatewidePercentage"]/avergaerankStatewidePercentage *10).fillna(0.0).astype(int)
school_high_pd["pg_avg_averageStandardScore"] = (school_high_pd["averageStandardScore"]/avergaeaverageStandardScore *10).fillna(0.0).astype(int)
school_high_pd["pg_avg_rankMovement"] = school_high_pd["rankMovement"]/avergaerankMovement *10
school_high_pd["pg_avg_pupilTeacherRatio"] = (school_high_pd["pupilTeacherRatio"]/avergaepupilTeacherRatio *10).fillna(0.0).astype(int)

school_high_pd['pt_rankStars'] = school_high_pd.apply(lambda row: f(row['pg_avg_rankStars']), axis=1)
school_high_pd['pt_rankStatewidePercentage'] = school_high_pd.apply(lambda row: f(row['pg_avg_rankStatewidePercentage']), axis=1)
school_high_pd['pt_averageStandardScore'] = school_high_pd.apply(lambda row: f(row['pg_avg_averageStandardScore']), axis=1)
school_high_pd['pt_rankMovement'] = school_high_pd.apply(lambda row: g(row['rankMovement']), axis=1)
school_high_pd['pt_pupilTeacherRatio'] = school_high_pd.apply(lambda row: f(row['pg_avg_pupilTeacherRatio']), axis=1)
school_high_pd['total_points'] = school_high_pd.apply(lambda row: h(row),axis=1)

school_high_points = school_high_pd[['postal','total_points']]
high_groupby = school_high_points.groupby('postal').mean()
high_groupby

In [ ]:
zip_final_school_score = []

combined_df = pd.merge(high_groupby, middle_groupby, how='outer', on='postal')

zip_final_school_score = pd.merge(combined_df, elementary_groupby, how='outer', on='postal')
zip_final_school_score.replace(np.nan, 0, inplace=True)
zip_final_school_score.rename(columns={'total_points_x': 'high_score','total_points_y': 'middle_score', 'total_points': 'elementary_score'}, inplace=True)
zip_final_school_score['final_score'] = zip_final_school_score.apply(lambda row: i(row), axis=1)
zip_final_school_score

# Price Analysis

In [ ]:
# get properties for each zip code (10 queries/min)
url = "https://search.onboard-apis.com/propertyapi/v1.0.0/property/address?"
property_results = []
prop_types = ['RESIDENTIAL (NEC)', #, 'CONDOMINIUM', 
              'APARTMENT'] #, 'CABIN', 'DUPLEX', 
              #'MANUFACTURED HOME', 'MOBILE HOME',
              #'TRIPLEX', 'TOWNHOUSE/ROWHOUSE']

sys.stdout.write('working..')

for z in latlngs:
    for pt in prop_types:
        params = {
            "postalcode": z['zipcode'],
            "propertytype": pt,
            "pagesize": "100"
        }

        res = fetch(url, params)

        property_results.append(res)
        wait(8)

In [ ]:
# get attomIds from property results
properties = []
attomIds = []

for p in property_results:
    for q in p['property']:
        properties.append(q)

props_df = pd.DataFrame(properties)
attomId_series = props_df['identifier']

for a in attomId_series:
    attomIds.append(a['attomId'])

In [ ]:
# get home values for each property (10 queries/min)
url = "https://search.onboard-apis.com/propertyapi/v1.0.0/property/basicprofile?"
sys.stdout.write("working..")
hv_results = []

for a in attomIds:
    params = {
        "attomId": a,
        "page":"1",
        "pagesize":"100"
    }

    idres = fetch(url, params)
    hv_results.append(idres)    
    wait(10)

In [ ]:
prop_res = pd.DataFrame(hv_results)
props = prop_res['property']
zcmv = []

for i in range(len(props)):
    zcmv.append({
        "zipcode":props[i][0]['address']['postal1'], 
        "avg_mkt_val":int(props[i][0]['assessment']['market']['mktTtlValue'])
    })

zips_mktvals_df = pd.DataFrame(zcmv).set_index('zipcode')
mkt_avg = zips_mktvals_df.groupby('zipcode').sum()/len(props)

mkt_avg_df = pd.DataFrame(mkt_avg)
mkt_avg_df = mkt_avg_df.reset_index()
mkt_avg_df.style.format({'avg_mkt_val':'${:,.2f}'})